In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import yaml
config = yaml.load(open('../src/configs/config.yaml', 'r'))

/home/francisco/anaconda3/envs/covid/lib/python3.6/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [2]:
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

In [3]:
df = pd.read_csv('data/raw/br_cases_city_daily_notification_rate.csv', index_col=0)
df.query('state_id == "CE"') 

FileNotFoundError: [Errno 2] File data/raw/br_cases_city_daily_notification_rate.csv does not exist: 'data/raw/br_cases_city_daily_notification_rate.csv'

In [4]:
social_dist_df = pd.read_csv('data/raw/Social Distancing Index by States.csv', index_col=0)
social_dist_df.query('state_name == "Ceará"') 


,isolated,dt
state_name,,
Ceará,0.501520,2020-05-14
Ceará,0.508907,2020-05-13
Ceará,0.506674,2020-05-12
Ceará,0.503496,2020-05-11
Ceará,0.536066,2020-05-10
...,...,...
Ceará,0.285186,2020-02-05
Ceará,0.283145,2020-02-04
Ceará,0.262221,2020-02-03


In [5]:
#year = 2018
#types = {
#    'States': geobr.read_state(code_state='all', year=year)
#}

Sorting the data by states alphabetical order

In [6]:
#geodf = types['States']
#geodf = geodf.sort_values('name_state').reset_index(drop=True)

In [7]:
def _df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

In [8]:
t = social_dist_df.reset_index()\
                      .pivot(index='state_name', 
                              columns='dt', 
                              values='isolated')\
                       .fillna(0)\
                       .dropna(how='all')

In [9]:
t

dt,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14
state_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Acre,0.362524,0.422642,0.296582,0.305246,0.311875,0.313577,0.272835,0.313434,0.401316,0.269315,0.266297,0.274479,0.293770,0.294643,0.377479,0.434939,0.270582,0.302516,0.336494,0.289538,0.288433,0.358979,0.427710,0.375058,0.437039,0.375934,0.311966,0.296750,0.347828,0.463544,0.290673,0.318144,0.345834,0.347295,0.331057,0.363795,0.451970,0.329315,0.338222,0.325418,0.339943,0.358175,0.404215,0.470485,0.342790,0.342630,0.417786,0.434641,0.448473,0.538698,0.632235,0.541323,0.552646,0.542710,0.531844,0.507685,0.548979,0.587434,0.505738,0.481963,0.465330,0.493927,0.484397,0.511080,0.578378,0.468820,0.462754,0.459697,0.463423,0.540488,0.486463,0.555869,0.466245,0.473580,0.518983,0.438476,0.452676,0.487781,0.565573,0.470766,0.534359,0.460119,0.467896,0.456944,0.486949,0.561649,0.460946,0.454715,0.435554,0.431786,0.513997,0.468990,0.540187,0.456057,0.455909,0.462108,0.464997,0.461440,0.481483,0.512476,0.470206,0.479756,0.475013,0.505890
Alagoas,0.314066,0.379520,0.241381,0.243551,0.252338,0.242239,0.213185,0.271564,0.366101,0.212346,0.224289,0.226968,0.239403,0.224909,0.314185,0.393054,0.225573,0.257906,0.276154,0.253829,0.237422,0.315328,0.403036,0.372282,0.419532,0.356466,0.263947,0.256214,0.297006,0.410717,0.257258,0.275945,0.291631,0.286947,0.273699,0.305304,0.406882,0.270028,0.285546,0.275141,0.297084,0.294733,0.354271,0.428260,0.306414,0.306438,0.352550,0.352270,0.359402,0.497439,0.587035,0.521321,0.525407,0.518176,0.514929,0.516706,0.518471,0.602396,0.468189,0.463480,0.445358,0.473059,0.454867,0.471169,0.536287,0.455897,0.441403,0.418389,0.420014,0.478865,0.453982,0.510215,0.431690,0.431221,0.437961,0.405937,0.405726,0.436681,0.523726,0.426333,0.495017,0.422451,0.415627,0.418337,0.438415,0.541858,0.427810,0.424954,0.413127,0.420388,0.477962,0.424488,0.476707,0.424055,0.421097,0.428257,0.439929,0.405466,0.431617,0.482373,0.438888,0.450799,0.449730,0.443570
Amapá,0.328219,0.393595,0.260747,0.333012,0.277639,0.277302,0.253864,0.305512,0.395825,0.227636,0.245081,0.249100,0.269423,0.270348,0.337753,0.409790,0.245358,0.275278,0.309688,0.267435,0.260665,0.333044,0.413045,0.361061,0.441496,0.382863,0.276604,0.268045,0.322080,0.439633,0.279351,0.307177,0.320773,0.313850,0.300555,0.328708,0.426171,0.306224,0.307445,0.294024,0.304655,0.311585,0.364094,0.465350,0.329309,0.341756,0.381565,0.467289,0.439093,0.565409,0.631642,0.541929,0.555323,0.540046,0.540150,0.522843,0.539120,0.586742,0.502756,0.493201,0.463889,0.489618,0.486944,0.499045,0.564747,0.477604,0.469513,0.473359,0.461159,0.568443,0.482879,0.562415,0.483556,0.479923,0.471414,0.454894,0.447479,0.478259,0.563691,0.463148,0.498764,0.456722,0.457237,0.443082,0.477390,0.571473,0.461828,0.465964,0.477494,0.448490,0.541510,0.483110,0.545409,0.478708,0

In [10]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(
    x=t.columns,
    y=t.query('state_name == "Ceará"').values[0],
    name='Ceará'))

# Add traces
fig.add_trace(go.Scatter(
    x=t.columns,
    y=t.query('state_name == "São Paulo"').values[0],
    name='São Paulo'))

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="Ceará",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "Distanciamento social",
                            "annotations": []}]),
                dict(label="São Paulo",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "Distanciamento social",
                            "annotations": []}]),
                dict(label="CE e SP",
                     method="update",
                     args=[{"visible": [True, True]},
                           {"title": "Distanciamento social",
                            "annotations": []}]),

            ]),
        )
    ])



fig.show()

In [11]:
states_name = list(t.index)

In [12]:
s = states_name[0]
f'state_name == "{ s }"'

'state_name == "Acre"'

In [13]:
t.query('state_name == "Ceará"')

dt,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14
state_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ceará,0.325898,0.38648,0.262221,0.283145,0.285186,0.267983,0.23068,0.282454,0.370019,0.231095,0.24935,0.249538,0.267707,0.247281,0.328566,0.396502,0.253475,0.28231,0.303254,0.262858,0.254861,0.341894,0.428128,0.396357,0.445741,0.371014,0.286426,0.27947,0.313335,0.430039,0.287555,0.30587,0.325558,0.316617,0.298437,0.331214,0.418632,0.311181,0.32392,0.310172,0.333002,0.335723,0.379343,0.445982,0.329716,0.340067,0.376042,0.477516,0.43812,0.557969,0.62954,0.558597,0.563575,0.595224,0.549436,0.542741,0.551217,0.593161,0.513472,0.512123,0.47453,0.503381,0.492554,0.516239,0.571716,0.489913,0.484281,0.470021,0.473467,0.544782,0.500047,0.556789,0.487533,0.480737,0.49065,0.449298,0.459641,0.48027,0.552211,0.474448,0.547457,0.470828,0.466689,0.456648,0.48628,0.542533,0.47501,0.463666,0.448059,0.446297,0.498189,0.455616,0.507437,0.468099,0.464888,0.47976,0.477048,0.486717,0.507584,0.536066,0.503496,0.506674,0.508907,0.50152


In [14]:
t.query(f'state_name == "{s}"').values

array([[0.36252444, 0.42264209, 0.29658191, 0.30524642, 0.31187468,
        0.31357698, 0.27283536, 0.31343361, 0.40131647, 0.26931473,
        0.26629738, 0.27447858, 0.29376987, 0.2946433 , 0.37747927,
        0.43493878, 0.27058217, 0.30251636, 0.33649404, 0.28953796,
        0.28843345, 0.35897942, 0.42770959, 0.375058  , 0.43703892,
        0.3759343 , 0.31196644, 0.29674995, 0.3478282 , 0.46354449,
        0.29067256, 0.31814444, 0.34583374, 0.34729511, 0.33105655,
        0.3637946 , 0.45196952, 0.32931529, 0.33822168, 0.32541785,
        0.33994253, 0.3581748 , 0.40421465, 0.47048518, 0.3427904 ,
        0.34263044, 0.41778588, 0.43464146, 0.44847292, 0.5386985 ,
        0.63223517, 0.54132313, 0.55264575, 0.54270957, 0.53184413,
        0.50768463, 0.54897949, 0.58743418, 0.50573758, 0.48196305,
        0.46532999, 0.49392693, 0.48439727, 0.51108019, 0.57837838,
        0.46881965, 0.46275406, 0.45969677, 0.4634229 , 0.54048795,
        0.48646315, 0.55586907, 0.46624452, 0.47

In [15]:
state_name = list(t.index)

c = 0
fig2 = go.Figure()
N=100
#loop to add all states to de scatter
for s in state_name:
    # Add traces
    fig2.add_trace(go.Scatter(
        x=t.columns,
        y=t.query(f'state_name == "{s}"').values[0],
        visible='legendonly',
        marker=dict(color=np.random.randn(N),
                   colorscale='Viridis'),
        name=f'{s}'))


fig2.show()


In [ ]:
df1 = social_dist_df.query('state_name == "Ceará"') 
df1